# Level 2

We have discussed [Level 1](onboarding_level1) in great detail. As we have seen, Level 1 is relatively simple.

In Level 2, we see more flexibility and additional choices. You will be exposed to some simple Python code and see how Aviary establishes a problem from beginning to end. Each step is built upon the capabilities of [Level 3](onboarding_level3). Users will be led to Level 3 in a natural way.

In [ ]:
# Testing Cell
import aviary.api as av
from aviary.api import Dynamic, Mission
from aviary.interface.methods_for_level1 import run_aviary
from aviary.utils.doctape import (
    get_variable_name,
    glue_function_arguments,
    glue_keys,
    glue_variable,
)

current_glued_vars = []

file_path = av.get_path('interface/methods_for_level1.py').relative_to(av.top_dir)
glue_variable(file_path, md_code=True)
glue_variable(file_path.name, md_code=True)

glue_variable(run_aviary.__name__ + '()', md_code=True)
AviaryValues = av.AviaryValues
glue_variable(AviaryValues.__name__, md_code=True)

# glue all argument of function run_aviary()
glue_function_arguments(run_aviary, current_glued_vars, glue_default=True, md_code=True)

glue_keys(dict(av.ProblemType.__members__))
ProblemType = av.ProblemType
glue_variable(ProblemType.__name__, md_code=True)

glue_variable(get_variable_name(Mission.Objectives.FUEL), md_code=True)
glue_variable(get_variable_name(Mission.Objectives.RANGE), md_code=True)
glue_variable(get_variable_name(Dynamic.Vehicle.MASS), md_code=True)

glue_variable('HEIGHT_ENERGY', av.EquationsOfMotion.HEIGHT_ENERGY.name, md_code=True)
glue_variable(
    'TWO_DEGREES_OF_FREEDOM', av.EquationsOfMotion.TWO_DEGREES_OF_FREEDOM.name, md_code=False
)
glue_variable('SOLVED_2DOF', av.EquationsOfMotion.SOLVED_2DOF.name, md_code=True)

glue_variable('height_energy', av.EquationsOfMotion.HEIGHT_ENERGY.value, md_code=True)
glue_variable('2DOF', av.EquationsOfMotion.TWO_DEGREES_OF_FREEDOM.value, md_code=True)
glue_variable('solved_2DOF', av.EquationsOfMotion.SOLVED_2DOF.value, md_code=True)

glue_variable('problem_type', av.Settings.PROBLEM_TYPE.split(':')[1], md_code=True)

In [ ]:
# Testing Cell
import aviary.api as av
from aviary.utils.doctape import glue_variable, run_command_no_file_error
from aviary.utils.functions import get_path

file_path = 'models/aircraft/test_aircraft/aircraft_for_bench_GwGm.csv'

command = 'aviary run_mission ' + file_path
glue_variable(command, md_code=True)
command += ' --max_iter 0 --optimizer IPOPT'
run_command_no_file_error(command)

file_path = get_path(file_path)
glue_variable(file_path.name, md_code=True)
glue_variable(file_path.stem, md_code=True)

phase_info_path = av.get_path('models/missions/two_dof_default.py').relative_to(av.top_dir.parent)
glue_variable('phase_info_path', phase_info_path, md_code=True)

Level 2 is defined in the [aviary/interface/methods_for_level1.py file](https://github.com/OpenMDAO/Aviary/blob/main/aviary/interface/methods_for_level1.py) and it has a method {glue:md}`run_aviary()` with a few arguments. If you examine {glue:md}`interface/methods_for_level1.py` you see that Level 1 prepares those arguments and then call {glue:md}`run_aviary()`. For {glue:md}`aviary run_mission models/aircraft/test_aircraft/aircraft_for_bench_GwGm.csv` examples, those arguments and their values are:

- {glue:md}`aircraft_data`: {glue:md}`aircraft_for_bench_GwGm.csv`
- {glue:md}`phase_info`: not provided (and will be loaded from {glue:md}`phase_info_path`)
- {glue:md}`optimizer`: {glue:md}`optimizer_default`
- {glue:md}`objective_type`: {glue:md}`objective_type_default`
- {glue:md}`record_filename`: {glue:md}`record_filename_default`
- {glue:md}`restart_filename`: {glue:md}`restart_filename_default`)
- {glue:md}`max_iter`: {glue:md}`max_iter_default`
- {glue:md}`run_driver`: {glue:md}`run_driver_default`
- {glue:md}`make_plots`: {glue:md}`make_plots_default`
- {glue:md}`phase_info_parameterization`: {glue:md}`phase_info_parameterization_default`
- {glue:md}`optimization_history_filename`: {glue:md}`optimization_history_filename_default`
- {glue:md}`verbosity`: {glue:md}`verbosity_default`

All the above arguments are straightforward except {glue:md}`objective_type`. Even though {glue:md}`objective_type` is `None`, it is not treated as `None`. In this scenario, the objective is set based on {glue:md}`problem_type` when using the {glue:md}`2DOF` mission method. There are three options for {glue:md}`problem_type` which is set to {glue:md}`SIZING` as default when aircraft is created. Aviary has the following mapping when user does not set {glue:md}`objective_type` but set `mission_method` to {glue:md}`2DOF` (in .csv file):

| {glue:md}`problem_type` | `objective` |
| ------------ | --------- |
| {glue:md}`SIZING` | {glue:md}`Mission.Objectives.FUEL` |
| {glue:md}`ALTERNATE` | {glue:md}`Mission.Objectives.FUEL`|
| {glue:md}`FALLOUT` | {glue:md}`Mission.Objectives.RANGE` |

In [ ]:
# Testing Cell
import openmdao.api as om

from aviary.api import Mission
from aviary.interface.methods_for_level2 import AviaryProblem
from aviary.mission.two_dof_problem_configurator import TwoDOFProblemConfigurator
from aviary.utils.doctape import check_contains, glue_class_functions, glue_variable
from aviary.variable_info.enums import EquationsOfMotion as EOM
from aviary.variable_info.enums import ProblemType as PT

EOM.HEIGHT_ENERGY
mo = Mission.Objectives
expected_objective = {PT.SIZING: mo.FUEL, PT.ALTERNATE: mo.FUEL, PT.FALLOUT: mo.RANGE}
for ptype, obj in expected_objective.items():
    dummy_prob = om.Problem()
    dummy_prob.mission_method = EOM.TWO_DEGREES_OF_FREEDOM
    dummy_prob.configurator = TwoDOFProblemConfigurator()
    dummy_prob.model = om.Group()
    dummy_prob.model.problem_type = ptype
    dummy_prob.model.target_range = 0
    AviaryProblem.add_objective(dummy_prob, verbosity=0)
    dummy_prob.setup()
    objectives = dummy_prob.model._responses.keys()
    check_contains(obj, objectives)

file_path = av.get_path('interface/methods_for_level2.py')
glue_variable(file_path.name, md_code=True)

# Get all functions of class AviaryProblem
glue_class_functions(AviaryProblem, current_glued_vars, pre_fix='prob', md_code=True)

# Get all functions of class TwoDOFProblemConfigurator
glue_class_functions(TwoDOFProblemConfigurator, current_glued_vars, pre_fix='prob', md_code=True)

In Aviary, {glue:md}`problem_type` is set to {glue:md}`SIZING` when it creates a vehicle (see [create_vehicle](https://github.com/OpenMDAO/Aviary/blob/main/aviary/utils/UI.py)). As you can see, since {glue:md}`problem_type` is {glue:md}`SIZING` by default in our case and we don't manually alter this setting, Aviary set objective to {glue:md}`Mission.Objectives.FUEL`. We will discuss more options of {glue:md}`objective_type` later on.

```{note}
If you want to use a custom objective function, you can set any arbitrary variable to be the objective by directly calling the OpenMDAO [`add_objective` method](https://openmdao.org/newdocs/versions/latest/features/core_features/adding_desvars_cons_objs/adding_objective.html#adding-an-objective) instead of using Aviary's built-in {glue:md}`add_objective()` method.
```

In our onboarding runs, we want to limit the number of iterations to 1 so that they all run faster. As a result, we will not consider whether the optimization converges. So, we will have

{glue:md}`max_iter`: 1

Level 2 cannot be run via `aviary` command on the command line. Users must develop level 2 Python code. The good news is that the Python code is pretty small. You can follow the following steps in order (we do not include function arguments for simplicity):

- `prob = AviaryProblem()`
- {glue:md}`prob.load_inputs()` 
- {glue:md}`prob.check_and_preprocess_inputs()`
- {glue:md}`prob.add_pre_mission_systems()`
- {glue:md}`prob.add_phases()`
- {glue:md}`prob.add_post_mission_systems()`
- {glue:md}`prob.link_phases()`
- {glue:md}`prob.add_driver()`
- {glue:md}`prob.add_design_variables()`
- {glue:md}`prob.add_objective()`
- {glue:md}`prob.setup()`
- {glue:md}`prob.set_initial_guesses()`
- {glue:md}`prob.run_aviary_problem()`

In the rest of this page, we will show a few examples to demonstrate how level 2 runs these steps. We start from rebuilding {glue:md}`aircraft_for_bench_GwGm` model in great details.

## Build level 2 for the same {glue:md}`aircraft_for_bench_GwGm` Model

We create a level 2 Python script to reproduce the {glue:md}`aircraft_for_bench_GwGm` model run that was used as an example in the level 1 document (this time we won’t use the level 1 functionality). The methods listed above are defined in level 3 (namely, [interface/methods_for_level2.py](https://github.com/OpenMDAO/Aviary/blob/main/aviary/interface/methods_for_level2.py)). You can run the code as follows:

In [ ]:
from copy import deepcopy

import aviary.api as av

# inputs that run_aviary() requires
aircraft_data = 'models/aircraft/test_aircraft/aircraft_for_bench_GwGm.csv'
optimizer = 'IPOPT'
objective_type = None
record_filename = 'aviary_history.db'
restart_filename = None
max_iter = 0
phase_info = deepcopy(av.default_2DOF_phase_info)

# Build problem
prob = av.AviaryProblem()

# Load aircraft and options data from user
# Allow for user overrides here
prob.load_inputs(aircraft_data, phase_info)

# Preprocess inputs
prob.check_and_preprocess_inputs()

# adds a pre-mission group (propulsion, geometry, aerodynamics, and mass)
prob.add_pre_mission_systems()

# adds a sequence of core mission phases.
prob.add_phases()

# adds a landing phase
prob.add_post_mission_systems()

# Link phases and variables
prob.link_phases()

# adds an optimizer to the driver
prob.add_driver(optimizer, max_iter=max_iter)

# adds relevant design variables
prob.add_design_variables()

# Load optimization problem formulation
# Detail which variables the optimizer can control
prob.add_objective(objective_type=objective_type)

# setup the problem
prob.setup()

# set initial guesses of states and controls variables
prob.set_initial_guesses()

# run the problem we just set up
prob.run_aviary_problem(record_filename, restart_filename=restart_filename)

In [ ]:
# Testing Cell
from openmdao.utils.assert_utils import assert_near_equal

from aviary.utils.doctape import glue_variable
from aviary.variable_info.variable_meta_data import Mission

obj_fuel = prob.get_val(Mission.Objectives.FUEL)
exp_fuel = 5.27313333
assert_near_equal(obj_fuel[0], exp_fuel, 1e-5)
glue_variable('str_obj_fuel', str(obj_fuel), md_code=True)

In this code, you do the same import as {glue:md}`methods_for_level1.py` does and set the values of all the arguments in {glue:md}`run_aviary()`. Now we will go through each line in detail to explain each step:

## Dissection of level 2 for the same `aircraft_for_bench_GwGm` model

All the methods of `prob` object (including its creation) are defined in level 3 ({glue:md}`methods_for_level2.py`). We now look at each of them.

We add other inputs that {glue:md}`run_aviary()` requires:

In [ ]:
aircraft_data = 'models/aircraft/test_aircraft/aircraft_for_bench_GwGm.csv'
optimizer = 'IPOPT'
objective_type = None
record_filename = 'aviary_history.db'
restart_filename = None
max_iter = 1

prob = av.AviaryProblem()

In [ ]:
# Testing Cell
import inspect

import aviary.api as av
from aviary.interface.methods_for_level2 import AviaryProblem
from aviary.utils.doctape import get_attribute_name, glue_variable

# read source code of __init__() starting from line 'self.model = AviaryGroup()'
source_code = inspect.getsource(AviaryProblem.__init__)
src_init = '```\n'
new_line = ''
start_reading = False
for ch in source_code:
    if ch == '\n':
        new_line = new_line.strip()
        if 'AviaryGroup()' in new_line:
            start_reading = True
        if start_reading:
            src_init += new_line + '\n'
        new_line = ''
    else:
        new_line += ch

glue_variable('src_init', src_init, md_code=False)

Several objects are initialized in this step:

```{glue:md} src_init
:format: myst
```

{glue:md}`phase_info` is a user defined dictionary (in a Python file) that controls the profile of the mission to be simulated (e.g. climb, cruise, descent segments etc). The line

In [ ]:
# Testing Cell
from aviary.utils.doctape import check_value, glue_variable

from aviary.utils.functions import get_model

filename = 'aircraft_for_bench_GwGm.csv'
turbofan_file = 'models/engines/turbofan_23k_1.csv'
data_var_name = 'aircraft:engine:data_file'  # read its value from .csv file
# glue_variable(data_var_name, md_code=True)
with open(get_model(filename)) as f_in:
    lines = f_in.readlines()
    for sLine in lines:
        if data_var_name in sLine:
            check_value(sLine.strip(), 'aircraft:engine:data_file,' + turbofan_file + ',unitless')
            s = '```\n'
            s += sLine.strip()
            glue_variable(turbofan_file, md_code=True)
            glue_variable('engine_data_file_line', s, md_code=False)
            val_and_units = s.split(',', 1)[-1]
            glue_variable('models/engines/turbofan_28k.csv,unitless', val_and_units, md_code=False)
            break

In [ ]:
phase_info = deepcopy(av.default_2DOF_phase_info)
prob.load_inputs(aircraft_data, phase_info)

is a function that has a few tasks:

- Read aircraft deck file `aircraft_data`
- Read phase info file {glue:md}`phase_info`
- Build core subsystems

We have seen {glue:md}`aircraft_data` file (a `.csv` file) in our level 1 examples. In [level 1](onboarding_level1), we simply called it input file. An aircraft model can also be directly defined in Python, by setting up an {glue:md}`AviaryValues` object with the desired inputs and options normally found in an input file. That object can be provided in the place of {glue:md}`aircraft_data`.


Engines are built by using the input data {glue:md}`engine_data_file` in the .csv file. For example in {glue:md}`aircraft_for_bench_GwGm.csv` file, we see:

```{glue:md} engine_data_file_line
:format: myst
```

In [ ]:
# Testing Cell
import pandas as pd
from myst_nb import glue

from aviary.api import Aircraft
from aviary.subsystems.propulsion.engine_model import EngineModel
from aviary.utils.functions import get_path

glue_variable('engine_data_file', Aircraft.Engine.DATA_FILE)
glue_variable(EngineModel.__name__, md_code=True)

# build the top rows of turbofan_28k.csv from that file
file_path = get_path('models/engines/turbofan_28k.csv')
df = pd.read_csv(file_path, nrows=5, skiprows=2, header=0)
glue('top_rows_of_engine_deck', df)

So, {glue:md}`engine_data_file` has value {glue:md}`models/engines/turbofan_28k.csv,unitless`. We follow this path and open that file. The top rows of engine deck file are:

```{glue:figure} top_rows_of_engine_deck
:name: 'tbl:df'
```

Users can provide an {glue:md}`EngineModel` instance of their own to use in Aviary's propulsion systems.

Other subsystems, including mass, geometry, and aerodynamics, are set up according to which legacy code options the user has specified in their input file, using `settings:equations_of_motion` and `settings:mass_method`. Aerodynamics is set up to match the selected equations of motion, while geometry will use either GASP, FLOPS, or both methods as required to calculate all values needed by other subsystems.

Next we check the user-provided inputs:

In [ ]:
# Testing Cell
from aviary.api import Settings

Settings.EQUATIONS_OF_MOTION
Settings.MASS_METHOD

In [ ]:
prob.check_and_preprocess_inputs()

This method checks the user-supplied input values for any potential problems. These problems include variable names that are not recognized in Aviary, conflicting options or values, or units mismatching.

Next, we add pre-mission systems:

In [ ]:
prob.add_pre_mission_systems()

This call adds a pre-mission group which includes propulsion, geometry, aerodynamics, and mass subsystems.

Next is the line

In [ ]:
prob.add_phases()

which adds a sequence of core mission phases. In addition, if `mission_method` is {glue:md}`2DOF` and `ascent` is a phase, it adds an equality constraint to the problem to ensure that the velocity at the end of the groundroll phase is equal to the rotation velocity at the start of the rotation phase ({glue:md}`_add_groundroll_eq_constraint()`). If `mission_method` is {glue:md}`height_energy`, it sets up trajectory parameters by calling `setup_trajectory_params()`. If `mission_method` is {glue:md}`solved_2DOF`, it has a block of code to make sure that the trajectory is smooth by applying boundary constraints between phases (e.g. fuselage pitch angle or true airspeed).

For {glue:md}`height_energy` missions, aviary currently models FLOPS' "simplified" takeoff as defined in [mission/flops_based/phases/simplified_takeoff.py](https://github.com/OpenMDAO/Aviary/blob/main/aviary/mission/flops_based/phases/simplified_takeoff.py).

It follows by adding post-mission subsystems:

In [ ]:
prob.add_post_mission_systems()

Similar to pre-mission, it adds a landing phase if `include_landing` key of `post_mission` has value of `True`. If the user chooses to define a `post_mission`, it will override the default. For {glue:md}`2DOF` missions, landing is defined in [mission/gasp_based/ode/landing_ode.py](https://github.com/OpenMDAO/Aviary/blob/main/aviary/mission/gasp_based/ode/landing_ode.py).

The next line is

In [ ]:
prob.link_phases()

This is important for allowing each phase of flight to pass to the next without discontinuities in the parameters. Consider Dymos' [Aircraft Balanced Field Length Calculation](https://openmdao.github.io/dymos/examples/balanced_field/balanced_field.html) example. In that example, we see separate nonlinear boundary constraints, nonlinear path constraints, and phase continuity constraints between phases. We don't want to go deeper in this function call, but just point out that each individual link can be set via dymos function {glue:md}`link_phases()`. See [dymos API](https://openmdao.github.io/dymos/api/trajectory_api.html) for more details.

The code blocks in this function (namely, {glue:md}`link_phases()`) are for {glue:md}`2DOF`, {glue:md}`height_energy`, and {glue:md}`solved_2DOF` missions. The links are set up based on physical principals (e.g. you can’t have instantaneous changes in mass, velocity, position etc.). Special care is required if the user selects a different or unusual set of phases. 

Now, our aircraft and the mission are fully defined. We are ready to define an optimization problem. This is achieved by adding an optimization driver, adding design variables, and an objective. 

For {glue:md}`add_driver()` function, we accept its argument `use_coloring=None`. Coloring is a technique that OpenMDAO uses to compute partial derivatives efficiently. This will become important later.

In [ ]:
prob.add_driver(optimizer, max_iter=max_iter)

In [ ]:
# Testing Cell
from aviary.utils.doctape import glue_actions

glue_actions('run_mission', current_glued_vars, glue_default=True, glue_choices=True)

Available drivers for use in Aviary are {glue:md}`SLSQP`, {glue:md}`SNOPT`, and {glue:md}`IPOPT`. The table below summarizes the basic setting along with sample values (the settings are options required by each optimizer):

| **Optimizers** | **Drivers** | **Settings** |
| ---------- | ------- | -------- |
| {glue:md}`SNOPT` | om.pyOptSparseDriver() | `Major iterations limit`: 50<BR/>`Major optimality tolerance`: 1e-4<BR/>`Major feasibility tolerance`: 1e-6<BR/>`iSumm`: 6 |
| {glue:md}`IPOPT` | om.pyOptSparseDriver() | `tol`: 1e-9<BR/>`mu_init`: 1e-5<BR/>`max_iter`: 50<BR/>`print_level`: 5 |
| {glue:md}`SLSQP` | om.ScipyOptimizeDriver() | `tol`: 1.0E-9<BR/>`maxiter`: 50<BR/>`disp`: True |

Note that {glue:md}`SLSQP` is freely available, but its performance is not as good as {glue:md}`SNOPT` and {glue:md}`IPOPT` sometimes. {glue:md}`SNOPT` is a commercial optimizer, and it is free for academic use. {glue:md}`IPOPT` is an open-source optimizer and it is free for all users.

Design variables (and constraints) are set in the line {glue:md}`prob.add_design_variables()`:

In [ ]:
prob.add_design_variables()

In [ ]:
# Testing Cell
from aviary.variable_info.enums import EquationsOfMotion as EOM
from aviary.variable_info.enums import ProblemType as PT

EOM.HEIGHT_ENERGY
EOM.TWO_DEGREES_OF_FREEDOM
PT.SIZING
PT.ALTERNATE
PT.FALLOUT

The details of the design variables and constraints are in the source code of {glue:md}`add_design_variables()`. Let us summarize the data below:

For default {glue:md}`height_energy` mission model, it is relatively simple:

| **Design Variables** | **Lower Bound** | **Upper Bound** | **Reference Value** | **Units** |
| ----------- | ----------- | ----------- | --------------- | ----- |
| Mission.Design.GROSS_MASS | 10 | 900.e3| 175.e3 | lbm |

For default {glue:md}`2DOF` mission model, the design variables and constraints depend on the type of problems ({glue:md}`SIZING`, {glue:md}`ALTERNATE`, or {glue:md}`FALLOUT`, see {glue:md}`ProblemType` class in `aviary/variable_info/enums.py` for details). First, there are four common design variables and two common constraints. There are two more design variables and two constraints for sizing problems.

| **Problem Type** | **Design Variables** | **Lower Bound** | **Upper Bound** | **Reference Value** | **Units** |
| ----------- | ----------- | ----------- | ----------- | --------------- | ----- |
| Any | Mission.Takeoff.ASCENT_T_INITIAL | 0 | 100  | 30.0 | s |
| Any | Mission.Takeoff.ASCENT_DURATION  | 1 | 1000 | 10.0 | s |
| Any | tau_gear  | 0.01 | 1.0 | 1 | s |
| Any | tau_flaps | 0.01 | 1.0 | 1 | s |
| SIZING | Mission.Design.GROSS_MASS | 10. | None | 175_000 | lbm |
| SIZING | Mission.Summary.GROSS_MASS | 10. | None | 175_000 | lbm |
| ALTERNATE | Mission.Summary.GROSS_MASS | 0 | infinite | 175_000 | lbm |
| **Problem Type** | **Constraint** | **Relation** | **Value** | **Reference Value** | **Units** |
| Any | h_fit.h_init_gear | = | 50.0  | 50.0 | ft |
| Any | h_fit.h_init_flaps | = | 400.0 | 400.0 | ft |
| SIZING | Mission.Constraints.RANGE_RESIDUAL | = | 0 | 10 | unitless |
| ALTERNATE | Mission.Constraints.RANGE_RESIDUAL | = | 0 | 10 | lbm |

In the above table, there are two hard-coded design variables: `tau_gear` and `tau_flaps`. They represent fractions of ascent time to start gear retraction and flaps retraction. There are two hard-coded constraints: `h_fit.h_init_gear` and `h_fit.h_init_flaps`. They are the altitudes of initial gear retraction and initial flaps retraction. The underscore in number '175_000' is for readability only. 

There are other constraints using OpenMDAO's `EQConstraintComp` component. We will not go into the details as this part is complicated and needs special attention. Note that each subsystem (for example engine model) might have their own design variables (think, for example, sizing the engine). Aviary goes through all subsystems and adds appropriate design variables.

You can override all the functions in level 3. So, you can set up your constraints in level 3 if the above ones do not meet your requirements.

The optimization objective is added to the problem by this line:

In [ ]:
# Testing Cell
from aviary.interface.methods_for_level2 import AviaryProblem
from aviary.models.missions.two_dof_default import phase_info

prob = AviaryProblem()
from aviary.models.aircraft.large_single_aisle_1.V3_bug_fixed_IO import V3_bug_fixed_options
from aviary.utils.doctape import check_contains
from aviary.validation_cases.validation_tests import get_flops_inputs

prob.load_inputs('models/test_aircraft/aircraft_for_bench_GwGm.csv', phase_info, verbosity=1)

prob.aviary_inputs.set_val(Settings.VERBOSITY, 3)

prob.check_and_preprocess_inputs()
prob.add_pre_mission_systems()
prob.add_phases()
prob.add_post_mission_systems()
prob.link_phases()

prob.add_design_variables()

In [ ]:
prob.add_objective(objective_type=objective_type)

The selection of objective is a little complicated. 

Earlier in this page, we have discussed the objective when `objective_type=None` and `mission_method` is {glue:md}`2DOF`. Let us discuss the other situations.

There are several objective types that users can choose: `mass`, `hybrid_objective`, `fuel_burned`, and `fuel`. 

| objective_type | objective |
| -------------- | --------- |
| mass | {glue:md}`Dynamic.Vehicle.MASS` |
| hybrid_objective | `-final_mass / {takeoff_mass} + final_time / 5.` |
| fuel_burned | `initial_mass - mass_final` (for `height_energy` mission only)|
| fuel | {glue:md}`Mission.Objectives.FUEL` |

As listed in the above, if `objective_type="mass"`, the objective is the final value of {glue:md}`Dynamic.Vehicle.MASS` at the end of the mission.
If `objective_type="fuel"`, the objective is the {glue:md}`Mission.Objectives.FUEL`.
There is a special objective type: `hybrid_objective`. When `objective_type='hybrid_objective'`, the objective is a mix of minimizing fuel burn and minimizing the mission duration:

```
      obj = -final_mass / {takeoff_mass} + final_time / 5.
```
This is because if we just minimized fuel burn then the optimizer would probably fly the plane slowly to save fuel, but we actually care about some mix of minimizing fuel burn while providing a reasonable travel time for the passengers. This leads to the `hybrid_objective` which seeks to minimize a combination of those two objectives. `final_time` is the duration of the full mission and is usually in the range of hours. So, the denominator `5.` means `5 hours`. That's just a value to scale the final_time variable. Since it's a composite objective we didn't want to have OpenMDAO do the scaling because the two variables in the objective are of a different order of magnitude.

If `objective_type=None` for a {glue:md}`2DOF` mission, Aviary will choose the objective based on `mission_method` and {glue:md}`problem_type`. We have discussed this case earlier in this page.

**Note:**  Aviary variable `Mission.Objectives.FUEL` when using the {glue:md}`2DOF` mission is actually a hybrid objective defined as

```
      reg_objective = overall_fuel/10000 + ascent_duration/30.
```
where `overall_fuel` has the unit of `lbm` and `ascent_duration` has the unit of seconds. In our case, `settings:equations_of_motion = 2DOF`, the final value of objective is {glue:md}`str_obj_fuel`, with `ref: 1.0` and `units: blank`. The units should be interpreted as `unitless`.

Here, `ref` is the reference value. For different objectives, the range may vary significantly different. We want to normalize the value. Ideally, users should choose `ref` such that the objective is in the range of `(0,1)`. This is required by optimizer.

**Note:**  Unfortunately, not all `objective_type` and `mission_method` combinations work.

Next is a line to call

In [ ]:
prob.setup()

This is a lightly wrapped OpenMDAO {glue:md}`setup()` method for the problem. It allows us to do `pre-` and `post-setup` changes, like adding calls to `set_input_defaults` and do some simple `set_vals` if needed. 

If we look at the signature of {glue:md}`setup()` in OpenMDAO's [Problem](https://openmdao.org/newdocs/versions/latest/_srcdocs/packages/core/problem.html) class, we find that the available kwargs are: `check`, `logger`, `mode`, `force_alloc_complex`, `distributed_vector_class`, `local_vector_class`, and `derivatives`. The ones that Aviary uses are `check` and `force_alloc_complex`. Argument `check` is a flag to determine default checks are performed. [Default checks](https://openmdao.org/newdocs/versions/latest/theory_manual/setup_stack.html) are: 'auto_ivc_warnings', comp_has_no_outputs', 'dup_inputs', 'missing_recorders', 'out_of_order', 'solvers', 'system', 'unserializable_options'.

If [force_alloc_complex](https://openmdao.org/newdocs/versions/latest/advanced_user_guide/complex_step.html) is true, sufficient memory will be allocated to allow nonlinear vectors to store complex values while operating under complex step. For our example, we don't use any of them.

For optimization problems, initial guesses are important.

In [ ]:
prob.set_initial_guesses()

For {glue:md}`solved_2DOF` and {glue:md}`2DOF` missions, this method performs several calls to `set_val` on the trajectory for states and controls to seed the problem with reasonable initial guesses using `initial_guesses` within corresponding phases (e.g. `height_energy.py` and `two_dof.py`). For `solved_2DOF` missions, it performs similar tasks but for hard-coded state parameters. This is reasonable because a `solved_2DOF` mission is actually a level 3 Aviary approach. We will cover it in [level 3 onboarding doc](onboarding_level3) in the next page. Note that initial guesses for all phases are especially important for collocation methods.

The last line is to run the problem we just set up:

In [ ]:
prob.run_aviary_problem()

This is a simple wrapper of Dymos' [run_problem()](https://openmdao.github.io/dymos/api/run_problem_function.html) function. It allows the users to provide `record_filename`, `restart_filename`, `suppress_solver_print`, and `run_driver`. In our case, `record_filename` is changed to `aviary_history.db` and `restart_filename` is set to `None`. The rest of the arguments take default values. If a restart file name is provided, aviary (or dymos) will load the states, controls, and parameters as given in the provided case as the initial guess for the next run. We have discussed the `.db` file in [level 1 onboarding doc](onboarding_level1) and will discuss how to use it to generate useful output in [level 3 onboarding doc](onboarding_level3).

Finally, we can add a few print statements for the variables that we are interested:


In [ ]:
print('Mission.Objectives.FUEL', prob.get_val(Mission.Objectives.FUEL, units='unitless'))
print('Mission.Design.FUEL_MASS', prob.get_val(Mission.Design.FUEL_MASS, units='lbm'))
print(
    'Mission.Design.FUEL_MASS_REQUIRED',
    prob.get_val(Mission.Design.FUEL_MASS_REQUIRED, units='lbm'),
)
print('Mission.Summary.TOTAL_FUEL_MASS', prob.get_val(Mission.Summary.TOTAL_FUEL_MASS, units='lbm'))
print(
    'Mission.Summary.GROSS_MASS (takeoff_mass)',
    prob.get_val(Mission.Summary.GROSS_MASS, units='lbm'),
)
print(
    'Mission.Landing.TOUCHDOWN_MASS (final_mass)',
    prob.get_val(Mission.Landing.TOUCHDOWN_MASS, units='lbm'),
)
print()

print(
    'Groundroll Final Mass (lbm)',
    prob.get_val('traj.groundroll.states:mass', units='lbm')[-1],
)
print('Rotation Final Mass (lbm)', prob.get_val('traj.rotation.states:mass', units='lbm')[-1])
print('Ascent Final Mass (lbm)', prob.get_val('traj.ascent.states:mass', units='lbm')[-1])
print('Accel Final Mass (lbm)', prob.get_val('traj.accel.states:mass', units='lbm')[-1])
print('Climb1 Final Mass (lbm)', prob.get_val('traj.climb1.states:mass', units='lbm')[-1])
print('Climb2 Final Mass (lbm)', prob.get_val('traj.climb2.states:mass', units='lbm')[-1])
print(
    'Cruise Final Mass (lbm)',
    prob.get_val('traj.phases.cruise.rhs.calc_weight.mass', units='lbm')[-1],
)
print('Desc1 Final Mass (lbm)', prob.get_val('traj.desc1.states:mass', units='lbm')[-1])
print('Desc2 Final Mass (lbm)', prob.get_val('traj.desc2.states:mass', units='lbm')[-1])
print('done')

We will cover user customized outputs in [level 3](onboarding_level3).

## Level 2: Another example

We now use a similar aircraft, a large single aisle commercial transport aircraft, but with a different mass estimation and mission method. Let us run Aviary using this input deck in level 1 first.

In [ ]:
!aviary run_mission models/aircraft/test_aircraft/aircraft_for_bench_FwFm.csv --max_iter 0 --optimizer IPOPT

Once again, to convert it to a level 2 model, we need to set all the arguments in level 1 manually.

By running a model in level 2 directly, we have the flexibility to modify the input parameters (e.g. {glue:md}`phase_info`). Let us continue to make modifications and obtain a different run script shown below:

In [ ]:
phase_info = {
    'pre_mission': {
        'include_takeoff': False,
        'optimize_mass': False,
    },
    'cruise': {
        'subsystem_options': {'core_aerodynamics': {'method': 'computed'}},
        'user_options': {
            'num_segments': 2,
            'order': 3,
            'mach_optimize': False,
            'mach_polynomial_order': 1,
            'mach_initial': (0.72, 'unitless'),
            'mach_final': (0.72, 'unitless'),
            'mach_bounds': ((0.7, 0.74), 'unitless'),
            'altitude_optimize': False,
            'altitude_polynomial_order': 1,
            'altitude_initial': (35000.0, 'ft'),
            'altitude_final': (35000.0, 'ft'),
            'altitude_bounds': ((23000.0, 38000.0), 'ft'),
            'throttle_enforcement': 'boundary_constraint',
            'time_initial_bounds': ((0.0, 0.0), 'min'),
            'time_duration_bounds': ((10.0, 30.0), 'min'),
        },
        'initial_guesses': {'time': ([0, 30], 'min')},
    },
    'post_mission': {
        'include_landing': False,
    },
}

# inputs that run_aviary() requires
aircraft_data = 'models/aircraft/test_aircraft/aircraft_for_bench_FwFm.csv'
mission_method = 'height_energy'
mass_method = 'FLOPS'
optimizer = 'SLSQP'
objective_type = None
record_filename = 'history.db'
restart_filename = None

# Build problem
prob = av.AviaryProblem()

# Load aircraft and options data from user
# Allow for user overrides here
prob.load_inputs(aircraft_data, phase_info)

# Preprocess inputs
prob.check_and_preprocess_inputs()

prob.add_pre_mission_systems()

prob.add_phases()

prob.add_post_mission_systems()

# Link phases and variables
prob.link_phases()

prob.add_driver(optimizer, max_iter=0)

prob.add_design_variables()

# Load optimization problem formulation
# Detail which variables the optimizer can control
prob.add_objective(objective_type=objective_type)

prob.setup()

prob.set_initial_guesses()

prob.run_aviary_problem(record_filename)

print('done')

As you see, there is a single phase `cruise`, no takeoff, no landing. Note that we must set `include_takeoff` to `False` because Aviary internally tries to connect takeoff to climb phase which we don't provide. There should be a check to see if both takeoff and climb phase exist first. Aviary still has many things to be improved.

We will see more details for what users can do in [level 3](onboarding_level3).

Level 2 is where you can integrate user-defined [external subsystems](../user_guide/subsystems), which is one of the main features of the Aviary tool. [Examples](../user_guide/using_external_subsystems) of external subsystems are: acoustics, battery modeling, etc.
Assume that you already have an external subsystem that you want to incorporate it into your model. We show how to add external subsystems via `external_subsystems` key in {glue:md}`phase_info`.

We will cover external subsystems in details in [Models with External Subsystems](onboarding_ext_subsystem) page.


## Summary

As you see, level 2 is more flexible than level 1. In level 2, you can:
- add/remove pre-defined mission phases (via {glue:md}`phase_info`, see example above);
- scale design variables (via reference value in {glue:md}`phase_info`)
- import additional files (e.g. `aero_data`)
- set pre-defined objective (e.g. `hybrid_objective`)
- add external subsystems (via {glue:md}`phase_info`)
- set `use_coloring` (see example above).

Most Aviary users should be well-served by Level 2; we have purposefully constructed it to be capable of most all use cases, even those on the forefront of research in aircraft design.

That being said, there are some cases where Level 2 is not sufficient and you may need additional flexibility. We are ready to move on to [Level 3](onboarding_level3).